In [55]:
import sklearn
import time
import pandas as pd   
from matplotlib import pyplot
from numpy import mean
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split, RepeatedKFold, GridSearchCV, cross_val_score
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import GaussianNB, CategoricalNB
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2  # Choose an appropriate scoring function for your data
from sklearn.linear_model import LinearRegression


import numpy as np



In [56]:
df_train_orig = pd.read_csv('train.csv')
df_test_orig = pd.read_csv('test.csv')


In [57]:

print(df_train_orig.shape)
print(df_test_orig.shape)


(181507, 272)
(77789, 272)


In [58]:
df_train = df_train_orig.copy()
df_test = df_test_orig.copy()

In [59]:
target_col = 'price_doc'
row_id_col = 'row ID'

In [60]:
df_train.drop(['sub_area'], axis=1, inplace=True)
df_test.drop(['sub_area'], axis=1, inplace=True)

In [61]:
X_train = pd.get_dummies(df_train.drop(columns=[target_col]), drop_first=True)
X_test = pd.get_dummies(df_test, drop_first=True)


# train_columns = X_train.columns.tolist()

In [62]:
row_ids = df_test['row ID']
X_test.drop(['row ID'], axis=1, inplace=True)

In [63]:
print(X_train.shape,'\n', X_test.shape)

(181507, 273) 
 (77789, 273)


In [64]:
X = pd.get_dummies(df_train.drop(columns=[target_col]), drop_first=True)
y = df_train[target_col]



In [ ]:
from sklearn.feature_selection import SequentialFeatureSelector

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)


In [ ]:

# Define the linear regression model
lr = LinearRegression()

# Forward Feature Selection
sfs = SequentialFeatureSelector(lr, n_features_to_select=30, direction='forward', scoring='neg_mean_squared_error', n_jobs=3)
sfs.fit(X_train, y_train)

# Get the mask of selected features
selected_features = sfs.get_support()

In [ ]:
selected_col = X_train.columns[selected_features]
selected_col

# X_train = X_train[selected_col]
X_train.shape


In [ ]:
from sklearn.metrics import mean_squared_error

X_train_selected = X_train[selected_col]
X_val_selected = X_val[selected_col]

# Scale the selected features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_selected)
X_val_scaled = scaler.transform(X_val_selected)

# Train the linear regression model on the selected features
model = LinearRegression()
model.fit(X_train_scaled, y_train)

# Evaluate the model
predictions = model.predict(X_val_scaled)
rmse = np.sqrt(mean_squared_error(y_val, predictions))
print("RMSE with selected features:", rmse)

In [ ]:


X_test = X_test[selected_col]

X_test_scaled = scaler.transform(X_test)  # Assuming 'scaler' is the trained StandardScaler

# Make predictions on the test data
test_predictions = model.predict(X_test_scaled)

# Create a DataFrame with 'row ID' and predictions
result_df = pd.DataFrame({'row ID': row_ids, 'price_doc': test_predictions})

# Save the results to a CSV file
result_df.to_csv('predictions3.csv', index=False)

In [ ]:

X_selected = X[selected_col]

# Scale the selected features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_selected)

# Train the linear regression model on the selected features
model = LinearRegression()
model.fit(X_scaled, y)